# FAST-UAV - Fixed-Wing Design Optimization
*Author: Félix Pollet - 2023* <br>

This notebook provides an example of design optimization of a fixed-wing UAV. The process is the same as the one for multirotor UAVs, except that some models and variables are different (e.g., for defining the wing geometry).

## 1. Setting up and analyzing the initial problem

In [ ]:
# Import required librairies
import os.path as pth
import sys
import logging
import fastoad.api as oad
import shutil
from fastuav.utils.postprocessing.analysis_and_plots import fixedwing_geometry_plot, mass_breakdown_sun_plot_drone

sys.path.append(pth.abspath("."))

#logging.basicConfig(level=logging.INFO, format="%(levelname)-8s: %(message)s")

# For using all screen width
from IPython.display import display, HTML, IFrame
display(HTML("<style>.container { width:95% !important; }</style>"))

In [ ]:
# Declare paths to folders and files
DATA_FOLDER_PATH = "data"
WORK_FOLDER_PATH = "workdir"
CONFIGURATION_FOLDER_PATH = pth.join(DATA_FOLDER_PATH, "configurations")
SOURCE_FOLDER_PATH = pth.join(DATA_FOLDER_PATH, "source_files")

CONFIGURATION_FILE = pth.join(CONFIGURATION_FOLDER_PATH, "fixedwing_mdo.yaml")
SOURCE_FILE = pth.join(SOURCE_FOLDER_PATH, "problem_inputs_FW.xml")  # Source file provided for the example

# For having log messages display on screen
#logging.basicConfig(level=logging.INFO, format="%(levelname)-8s: %(message)s")
#warnings.filterwarnings(action="ignore")

As you can see in the following N2 diagram, some models used for the fixed-wing UAV are (obviously) different from the multirotor ones. Have a look at the geometry discipline. The arms and body frame have been replaced by wings, tails, and a fuselage. The aerodynamics module also includes models for estimating the parasitic drag, among others.

In [ ]:
N2_FILE = pth.join(WORK_FOLDER_PATH, "n2.html")
oad.write_n2(CONFIGURATION_FILE, N2_FILE, overwrite=True)
from IPython.display import IFrame
IFrame(src=N2_FILE, width="100%", height="500px")

Once again, we generate a temporary input file for this notebook, so that the reference source file cannot be corrupted:

In [ ]:
input_file = oad.generate_inputs(CONFIGURATION_FILE, SOURCE_FILE, overwrite=True)

The [problem_inputs.xml](./workdir/problem_inputs.xml) file now includes parameters such as the material properties for the spars and the ribs of the wing. Take your time to check these parameters specific to fixed-wing UAVs.

In [ ]:
oad.variable_viewer(input_file)

## 2. Running an MDO

You can now run an optimization problem. Here again, some design variables and constraints defined in the [configuration file](./data/configurations/fixedwing_mdo.yaml) are specific to fixed-wing UAVs. For example:

```yaml
optimization:
  design_variables:
    - name: optimization:variables:geometry:wing:loading:k  # under estimation coefficient on the wing loading
      lower: 0.1
      upper: 1.0
    - name: optimization:variables:geometry:wing:AR  # aspect ratio of the wing
      lower: 8.0
      upper: 20.0
  constraints:
    - name: optimization:constraints:geometry:fuselage:volume  # fuselage volume requirement (to carry the batteries and the payload)
      lower: 0.0
```

In [ ]:
optim_problem = oad.optimize_problem(CONFIGURATION_FILE, overwrite=True)

Let's save and visualize these results:

In [ ]:
output_file = optim_problem.output_file_path
shutil.copy(output_file, 
            pth.join(SOURCE_FOLDER_PATH, 'problem_outputs_FW_mdo.xml')
)

In [ ]:
oad.optimization_viewer(CONFIGURATION_FILE)

In [ ]:
oad.variable_viewer(output_file)

## 3. Analysis and plots

You can now use postprocessing plots to visualize the results of the MDO.

In [ ]:
fig = fixedwing_geometry_plot(output_file)
fig.show()

In [ ]:
fig = mass_breakdown_sun_plot_drone(output_file)
fig.show()